In [8]:
from lane_detection import *
from lane_following import *

In [9]:
def show(img):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

In [10]:
import errno
import os
import signal
import functools

class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wrapper

    return decorator

In [11]:
@timeout(2)
def render_frame(frame):
    # Process image
    sliced = split(frame)
    height = sliced.shape[0]
    width = sliced.shape[1]
    gray = to_gray(sliced)
    blurred = to_blurred(gray)
    bw = to_bw(blurred)
    edges = find_edges(bw)
    lines = find_lines(edges)
    grouped_lines = group_lines(lines, height, slope_tolerance=0.1, x_intercept_tolerance=50) # group lines
    merged_lines = merge_lines(grouped_lines, height, width) # merge groups of lines
    lanes = detect_lanes(bw, merged_lines, 500, 200, 10)
    center_lines = merge_lane_lines(lanes, height) # find the center of each lane
    center_line = pick_center_line(center_lines, width) # find the closest lane
    (move, turn) = suggest_direction(center_line, width) # textual suggestion of how to move
    text = f"The AUV should move {move} and turn {turn}"
    frame = draw_lanes(frame, lanes, offset=True)
    frame = draw_lines(frame, [center_line], (0, 0, 255), offset=True)
    frame = cv2.putText(frame, text, (0, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
    return cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)



In [12]:
def render_frame_by_step(frame):
    sliced = split(frame)
    height = sliced.shape[0]
    width = sliced.shape[1]
    show(sliced)
    gray = to_gray(sliced)
    show(gray)
    blurred = to_blurred(gray)
    show(blurred)
    bw = to_bw(blurred, 85)
    show(bw)
    edges = find_edges(bw, 20, 100, 3)
    show(edges)
    lines = find_lines(edges, 1, np.pi/180, 150, 100, 20)
    show(draw_lines(frame, lines, offset=True))
    grouped_lines = group_lines(lines, height, slope_tolerance=0.1, x_intercept_tolerance=50) # group lines
    # for _, slope_group in grouped_lines.items():
    #     for _, x_group in slope_group.items():
    #         for line in x_group:
    #             print(f"{line}")
    #         show(draw_lines(frame, x_group, random=True, offset=True))
    merged_lines = merge_lines(grouped_lines, height, width) # merge groups of lines
    show(draw_lines(frame, merged_lines, offset=True))
    lanes = detect_lanes(bw, merged_lines, 500, 200, 10)
    show(draw_lanes(frame, lanes, offset=True))
    center_lines = merge_lane_lines(lanes, height) # find the center of each lane
    center_line = pick_center_line(center_lines, width) # find the closest lane
    (move, turn) = suggest_direction(center_line, width) # textual suggestion of how to move
    text = f"The AUV should move {move} and turn {turn}"
    frame = draw_lanes(frame, lanes, offset=True)
    frame = draw_lines(frame, [center_line], (0, 0, 255), offset=True)
    frame = cv2.putText(frame, text, (0, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
    show(draw_lines(frame, merged_lines, offset=True))

In [13]:
# frame = cv2.imread("frames/frame540.jpg")
# render_frame_by_step(frame)

In [14]:
cap = cv2.VideoCapture('AUV_Vid.mkv')

count = 0 # the number of frames since the last
frequency = 0 # the number of frames to skip
img_array = []

ret = True
while ret:
    ret, frame = cap.read()
    if count < 500:
        print(f"now on frame {count}...")
        try:
            frame = render_frame(frame)
        except Exception as e:
            print(f"failed to render frame {count}. Error: {e}")
            pass
        
        height, width, layers = frame.shape
        size = (width, height)
        img_array.append(frame)

    count += 1

cap.release()
print("Done processing frames. Now writing.")

out = cv2.VideoWriter("rendered_video.avi", cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

for img in img_array:
    out.write(img)
out.release()


now on frame 0...
now on frame 1...
now on frame 2...
now on frame 3...
now on frame 4...
now on frame 5...
now on frame 6...
now on frame 7...
now on frame 8...


now on frame 9...
now on frame 10...
now on frame 11...
now on frame 12...
now on frame 13...
now on frame 14...
now on frame 15...
now on frame 16...
now on frame 17...
now on frame 18...
now on frame 19...
now on frame 20...
now on frame 21...
now on frame 22...
now on frame 23...
now on frame 24...
now on frame 25...
now on frame 26...
now on frame 27...
now on frame 28...
now on frame 29...
now on frame 30...
now on frame 31...
now on frame 32...
now on frame 33...
now on frame 34...
now on frame 35...
now on frame 36...
now on frame 37...
now on frame 38...
now on frame 39...
now on frame 40...
now on frame 41...
now on frame 42...
now on frame 43...
now on frame 44...
now on frame 45...
now on frame 46...
now on frame 47...
now on frame 48...
now on frame 49...
now on frame 50...
now on frame 51...
now on frame 52...
now on frame 53...
now on frame 54...
now on frame 55...
now on frame 56...
now on frame 57...
now on frame 58...
now on frame 59...
now on frame 60...
now on frame 

/home/ebenq/Dev/python/cv-intro/lane_detection.py:275: RankWarning: Polyfit may be poorly conditioned
  m, b = np.polyfit(x_list, y_list, 1)  # line of best fit for the points
/home/ebenq/Dev/python/cv-intro/lane_detection.py:275: RankWarning: Polyfit may be poorly conditioned
  m, b = np.polyfit(x_list, y_list, 1)  # line of best fit for the points
/home/ebenq/Dev/python/cv-intro/lane_detection.py:275: RankWarning: Polyfit may be poorly conditioned
  m, b = np.polyfit(x_list, y_list, 1)  # line of best fit for the points


now on frame 91...
now on frame 92...
now on frame 93...
now on frame 94...
now on frame 95...
now on frame 96...
now on frame 97...
now on frame 98...
now on frame 99...
now on frame 100...
now on frame 101...
now on frame 102...
now on frame 103...
now on frame 104...
now on frame 105...
now on frame 106...
now on frame 107...
now on frame 108...
now on frame 109...
now on frame 110...
now on frame 111...
now on frame 112...
now on frame 113...
now on frame 114...
now on frame 115...
now on frame 116...
now on frame 117...
now on frame 118...
now on frame 119...
now on frame 120...
now on frame 121...
now on frame 122...
now on frame 123...
now on frame 124...
now on frame 125...
now on frame 126...
now on frame 127...
now on frame 128...
now on frame 129...
now on frame 130...
now on frame 131...
now on frame 132...
now on frame 133...
now on frame 134...
now on frame 135...
now on frame 136...
now on frame 137...
now on frame 138...
now on frame 139...
now on frame 140...
now on fr

/home/ebenq/Dev/python/cv-intro/lane_detection.py:275: RankWarning: Polyfit may be poorly conditioned
  m, b = np.polyfit(x_list, y_list, 1)  # line of best fit for the points


now on frame 159...
now on frame 160...
now on frame 161...
now on frame 162...
now on frame 163...
now on frame 164...
now on frame 165...
now on frame 166...
now on frame 167...
now on frame 168...
now on frame 169...
now on frame 170...
now on frame 171...
now on frame 172...
now on frame 173...
now on frame 174...
now on frame 175...
now on frame 176...
now on frame 177...
now on frame 178...
now on frame 179...
now on frame 180...
now on frame 181...
now on frame 182...
now on frame 183...
now on frame 184...
now on frame 185...
now on frame 186...
now on frame 187...
now on frame 188...
now on frame 189...
now on frame 190...
now on frame 191...
now on frame 192...
now on frame 193...
now on frame 194...
now on frame 195...
now on frame 196...
now on frame 197...
now on frame 198...
now on frame 199...
now on frame 200...
now on frame 201...
now on frame 202...
now on frame 203...
now on frame 204...
now on frame 205...
now on frame 206...
now on frame 207...
now on frame 208...


/home/ebenq/Dev/python/cv-intro/lane_detection.py:275: RankWarning: Polyfit may be poorly conditioned
  m, b = np.polyfit(x_list, y_list, 1)  # line of best fit for the points


now on frame 209...
now on frame 210...
now on frame 211...
now on frame 212...
now on frame 213...
now on frame 214...
now on frame 215...
now on frame 216...
now on frame 217...
now on frame 218...
now on frame 219...
now on frame 220...
now on frame 221...
now on frame 222...
now on frame 223...
now on frame 224...
now on frame 225...
now on frame 226...
now on frame 227...
now on frame 228...
now on frame 229...
now on frame 230...
now on frame 231...
now on frame 232...
now on frame 233...
now on frame 234...
now on frame 235...
now on frame 236...
now on frame 237...
now on frame 238...
now on frame 239...
now on frame 240...
now on frame 241...
now on frame 242...
now on frame 243...
now on frame 244...
now on frame 245...
now on frame 246...
now on frame 247...
now on frame 248...
now on frame 249...
now on frame 250...
now on frame 251...
now on frame 252...
now on frame 253...
now on frame 254...
now on frame 255...
now on frame 256...
now on frame 257...
now on frame 258...
